# Manutenzione Predittiva: Stima della Remaining Useful Life (RUL)

Questo notebook mostra un esempio di **manutenzione predittiva**. L'obiettivo è prevedere il momento in cui un componente meccanico si guasterà, in modo da pianificare interventi manutentivi ottimali.

Il processo si articola in **3 fasi**:

1. **Simulazione della curva di degrado** del componente nel tempo.
2. **Stima della Remaining Useful Life (RUL)** tramite tre approcci:
   - **Modello statistico a priori (Weibull)**: adatto in assenza totale di dati misurati.
   - **Modello intermedio (fit esponenziale)**: utilizza le **misurazioni fisiche parziali attuali** e informazioni dal **datasheet** del componente (es. soglia di guasto) per stimare la RUL, senza necessità di uno storico. Include anche un **intervallo di confidenza statistico** sulla stima.
   - **Modello a posteriori con confronto tra curve (DTW)**: richiede un archivio di curve di degrado storiche.
3. **Pianificazione dell'intervento**: identificazione del momento ottimale per la manutenzione, sulla base della RUL stimata.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from fastdtw import fastdtw
from scipy.optimize import curve_fit
from scipy.stats import t, weibull_min

## Stima RUL senza archivio storico: approccio statistico

Quando l'archivio di curve storiche non è ancora disponibile (ad esempio nelle prime fasi di monitoraggio di una macchina o di un componente), non è possibile applicare direttamente metodi di similitudine temporale come il DTW.

In questa fase iniziale, possiamo sfruttare:

- Parametri statistici derivati dai datasheet delle macchine (es. vita utile media, deviazione standard della durata),
- Modelli probabilistici semplici (es. distribuzioni di Weibull, esponenziale) che descrivono la durata attesa del componente,
- Indicatori di degrado basati su soglie di segnali fisici o di processo.

Questo approccio fornisce una stima grossolana della Remaining Useful Life (RUL), che può essere raffinata in seguito quando l’archivio dati sarà sufficientemente popolato.

In questo notebook, mostreremo un esempio semplice di stima statistica basata su una distribuzione parametrica tipica (Weibull), usando parametri ipotetici ricavati dal datasheet del componente.

La distribuzione di Weibull è un modello statistico ampiamente utilizzato per rappresentare i tempi di guasto o di vita utile di componenti industriali. 

Si caratterizza principalmente per il **parametro di forma** (shape), che descrive il comportamento del guasto nel tempo:
- se è < 1, il tasso di guasto diminuisce nel tempo (guasti precoci);
- se è = 1, il guasto avviene con tasso costante nel tempo (guasti casuali);
- se è > 1, il tasso di guasto aumenta nel tempo (usura).

Il **parametro di scala** (scale) definisce la durata tipica del componente.

Con la Weibull possiamo stimare la probabilità che un componente sopravviva fino a un certo tempo e quindi calcolare la **Remaining Useful Life (RUL)** come tempo residuo medio di vita, condizionata al fatto che il componente ha già funzionato per un certo intervallo di tempo.

Questo approccio è particolarmente utile quando non si dispone ancora di dati storici reali.


In [ ]:
def estimate_rul_statistical(current_time, shape=1.5, scale=1000):
    if current_time >= scale:
        return 0

    survival_prob = weibull_min.sf(current_time, c=shape, scale=scale)
    expected_life_total = weibull_min.mean(c=shape, scale=scale)

    expected_remaining_life = expected_life_total * survival_prob
    return max(0, expected_remaining_life - current_time)


# Esempio uso con current_time scelto
current_time_example = 150
shape = 1.5
scale = 1000
estimated_rul_stat = estimate_rul_statistical(current_time_example)

print(f"Tempo attuale di funzionamento: {current_time_example} unità di tempo")
print(f"Parametro di forma: {shape}")
print(f"Parametro di scala: {scale}")
print(f"RUL stimata (approccio statistico): {estimated_rul_stat:.2f} unità di tempo")

## Stima della RUL con approccio intermedio

Questo approccio intermedio si colloca tra i metodi puramente statistici (es. Weibull) e quelli data-driven avanzati (es. DTW o modelli di machine learning). L'obiettivo è stimare la RUL (Remaining Useful Life) di un componente anche in assenza di uno storico ampio, sfruttando:

- **Le misurazioni fisiche attuali** (es. vibrazione, temperatura);
- **Un modello empirico del degrado** (lineare, esponenziale, ecc.);
- **La soglia di guasto dichiarata** nel datasheet del componente.

Nel nostro esempio:
- Il componente è monitorato per vibrazione [mm/s];
- Il datasheet indica che **una vibrazione > 5 mm/s implica guasto imminente**;
- Abbiamo una serie di osservazioni recenti sul degrado del componente;
- Il trend appare **esponenziale** nel tempo.

Stimiamo la RUL proiettando il modello di degrado verso la soglia di guasto.


In [ ]:
# Parametri veri e simulazione osservazioni parziali
np.random.seed(42)
true_a, true_b = 1.0, 0.15
t_full = np.linspace(0, 20, 100)
y_true = true_a * np.exp(true_b * t_full)

# Osservazioni fino a t = 10
t_obs = t_full[t_full <= 10]
y_obs = true_a * np.exp(true_b * t_obs) + np.random.normal(0, 0.1, size=len(t_obs))


# Fit modello esponenziale
def exp_model(t, a, b):
    return a * np.exp(b * t)


params, cov = curve_fit(exp_model, t_obs, y_obs)
a_hat, b_hat = params
stderr = np.sqrt(np.diag(cov))

# Soglia di guasto
failure_threshold = 5.0

# Stima del tempo di guasto
t_failure_est = np.log(failure_threshold / a_hat) / b_hat
t_failure_true = np.log(failure_threshold / true_a) / true_b

# Intervallo di confidenza
alpha = 0.05
dof = len(t_obs) - len(params)
t_score = t.ppf(1 - alpha / 2, dof)

partial_a = -1 / (a_hat * b_hat)
partial_b = -np.log(failure_threshold / a_hat) / (b_hat**2)
var_t_failure = (
    (partial_a**2) * cov[0, 0]
    + (partial_b**2) * cov[1, 1]
    + 2 * partial_a * partial_b * cov[0, 1]
)
conf_interval = t_score * np.sqrt(var_t_failure)

# Predizione modello stimato
t_range = np.linspace(0, max(t_failure_est + 5, t_failure_true + 5), 200)
y_fit = exp_model(t_range, a_hat, b_hat)

# Visualizzazione
plt.figure(figsize=(10, 6))
plt.scatter(t_obs, y_obs, label="Osservazioni", color="blue")
plt.plot(t_range, y_fit, label="Fit esponenziale", color="green")
plt.axhline(failure_threshold, color="red", linestyle="--", label="Soglia guasto")

# Stime guasto e intervallo confidenza
plt.axvline(t_failure_est, color="orange", linestyle="--", label="Stima t_guasto")
plt.fill_betweenx(
    [0, failure_threshold],
    t_failure_est - conf_interval,
    t_failure_est + conf_interval,
    color="orange",
    alpha=0.3,
    label="Intervallo conf. 95%",
)

plt.xlabel("Tempo [giorni]")
plt.ylabel("Vibrazioni [mm/s]")
plt.title("Curva di degrado: modello stimato + intervallo confidenza")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Stima t_guasto: {t_failure_est:.2f} giorni")
print(
    f"Intervallo di confidenza 95%: [{t_failure_est - conf_interval:.2f}, {t_failure_est + conf_interval:.2f}] giorni"
)

## Stima della RUL basata su similarità temporale (DTW)

Una volta disponibile un archivio di curve storiche di degrado, è possibile stimare la RUL del componente corrente confrontandone la traiettoria di degradazione parziale con quelle archiviate.

Questo approccio si basa sul Dynamic Time Warping (DTW), una tecnica per misurare la somiglianza tra sequenze temporali anche se di lunghezza diversa o con andamento non lineare.

Nel notebook implementiamo un algoritmo che, data una curva parziale corrente, individua la curva più simile nell'archivio e usa la differenza di lunghezza per stimare la RUL residua.

Questo metodo è particolarmente utile per sistemi multivariati dove più feature sono monitorate contemporaneamente.

> **Nota:** anziché basarsi solo sulla curva storica più simile (*nearest neighbor*), è generalmente preferibile considerare le *k* curve più simili e stimare la RUL come media (o mediana) delle rispettive RUL residue. Questo permette di ridurre l’effetto di outlier o rumore in singole curve storiche, migliorando la robustezza della previsione.

### Simulazione e visualizzazione di curve di degrado su più segnali

Per testare l’approccio basato sul confronto tra andamenti temporali simili, generiamo un archivio sintetico di curve di degrado. Ogni curva rappresenta come cambia nel tempo lo stato di un componente, osservando più segnali contemporaneamente (ad esempio: vibrazioni, temperatura, consumo elettrico).

Questi segnali simulano il comportamento reale di una macchina che si degrada: inizialmente stabili, poi peggiorano progressivamente fino al guasto. Le curve complete serviranno da riferimento per stimare il tempo residuo di vita (RUL) confrontandole con dati parziali osservati in tempo reale.

In [ ]:
def generate_archive(num_curves=100, min_length=100, max_length=1000, n_features=3):
    archive = []
    for _ in range(num_curves):
        length = np.random.randint(min_length, max_length + 1)
        features = simulate_curve(length=length, n_features=n_features)
        archive.append(features)
    return archive


def plot_archive(archive):
    n_features = archive[0].shape[1]

    _, axs = plt.subplots(n_features, 1, figsize=(10, 3 * n_features), sharex=True)
    if n_features == 1:
        axs = [axs]

    for i in range(n_features):
        for curve in archive:
            axs[i].plot(curve[:, i], alpha=0.7)
        axs[i].set_title(f"Feature {i + 1}")
        axs[i].set_ylabel("Value")
    axs[-1].set_xlabel("Time")
    plt.tight_layout()
    plt.show()


def simulate_curve(length=500, n_features=3, explosion_start_ratio=0.8):
    t = np.linspace(0, 1, length)
    explosion_start = explosion_start_ratio

    features = []
    for _ in range(n_features):
        noise = 0.02 * np.random.randn(length)

        explosion = np.exp(t - explosion_start)
        explosion = explosion - explosion[int(explosion_start * length)]
        explosion = np.clip(explosion, 0, None)

        c, factor = np.random.uniform(0.1, 2), np.random.uniform(5, 15)
        feature_curve = np.exp(c * t + factor * explosion) + noise
        features.append(feature_curve)

    features = np.vstack(features).T
    return features


np.random.seed(42)

archive = generate_archive(100)
plot_archive(archive)

### Test della stima RUL basata su archivio

Ora che disponiamo di un archivio di curve storiche simulate, possiamo testare un approccio basato sulla similarità dinamica. In particolare:

- Generiamo una **curva parziale** rappresentante il comportamento corrente di un componente;
- Confrontiamo questa curva con tutte quelle nell'archivio tramite **Dynamic Time Warping (DTW)**;
- Identifichiamo la curva più simile nel comportamento iniziale;
- Stimiamo la **Remaining Useful Life (RUL)** come la differenza tra la durata completa della curva storica e il punto corrente della curva osservata.

Questo metodo sfrutta la forma complessiva della traiettoria per fornire una stima più realistica della RUL rispetto ad approcci puramente statistici, specialmente quando i dati reali iniziano ad accumularsi.

In [ ]:
current_time = np.random.randint(200, 400)
partial_curve = simulate_curve()[:][:current_time]

In [ ]:
def estimate_rul_by_similarity(archive, partial_curve):
    partial_len = len(partial_curve)
    best_dist = np.inf
    best_idx = -1
    for idx, curve in enumerate(archive):
        if len(curve) < partial_len:
            continue
        dist, _ = fastdtw(
            partial_curve[:partial_len], curve[:partial_len], dist=euclidean_dist
        )
        if dist < best_dist:
            best_dist = dist
            best_idx = idx
    estimated_rul = len(archive[best_idx]) - partial_len
    return estimated_rul, archive[best_idx], best_idx


def euclidean_dist(a, b):
    return np.linalg.norm(a - b)


estimated_rul, best_curve, best_idx = estimate_rul_by_similarity(archive, partial_curve)

print(f"Indice curva più simile: {best_idx}")
print(f"RUL stimata al timestep {len(partial_curve)}: {estimated_rul:.2f}")

In [ ]:
n_features = partial_curve.shape[1]

plt.figure(figsize=(12, 3 * n_features))
for i in range(n_features):
    plt.subplot(n_features, 1, i + 1)
    plt.title(f"Feature {i} - Curva corrente vs Curva più simile")
    plt.plot(partial_curve[:, i], "-o", label="Curva corrente")
    plt.plot(best_curve[:, i], label="Curva più simile")
    plt.legend()
plt.tight_layout()
plt.show()

### 📝 Conclusioni

In questo notebook abbiamo esplorato un esempio semplice e didattico di manutenzione predittiva, evidenziando tre diversi approcci per la stima della Remaining Useful Life (RUL):

- Il modello statistico basato sulla distribuzione di Weibull, utile in assenza di dati misurati.
- Un modello intermedio, che sfrutta misurazioni fisiche attuali e informazioni di progetto (datasheet), fornendo una stima della RUL con intervallo di confidenza, anche in assenza di dati storici estesi.
- Il modello a posteriori basato sul confronto dinamico di curve di degrado (DTW), efficace quando è disponibile un archivio di dati storici simili.

Per applicazioni reali, sarà necessario calibrare e validare i modelli con dati specifici e valutare l’integrazione in sistemi di monitoraggio continuo.